<a href="https://colab.research.google.com/github/Harrison613/connectProject/blob/main/Rolling_Thunder_LLM_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U langchain
!pip install -q -U accelerate
!pip install -q -U PyPDF2
!pip install -q -U faiss-gpu
!pip install -q -U sentence_transformers
!pip install -q -U pypdf
!pip install -q torch==2.1.0

In [2]:
from langchain.document_loaders import PyPDFLoader

#path to pdfs
path = "/content/drive/MyDrive/Technical Manuals" #download pdfs u wanna test to your drive and mount ur drive using the button on the left under the files(folder button)

#load pdf
reader = PyPDFLoader('/content/drive/MyDrive/Technical Manuals/AFD-180201-00-5-3.pdf')

#split pdf into pages
pages = reader.load_and_split()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=15,
    separators=['\n\n', '\n', '(?=>\. )', ' ', '']
    )

# split pages into text chunks
texts = text_splitter.split_documents(pages)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

#embedding text
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cuda'})

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, embeddings)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded", quantization_config=bnb_config, trust_remote_code=True) #LLM Model

tokenizer = AutoTokenizer.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded")

(…)nstruct-sharded/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)ded/resolve/main/configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)-sharded/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)arded/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)ruct-sharded/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)ded/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [8]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens=1024,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [9]:
from langchain import HuggingFacePipeline

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [10]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                 retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
                                 return_source_documents=True,
                                 verbose=False,
)

In [14]:
query = "What is a technical order?"
# Send question as a query to qa chain
result = qa({"query": query})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [15]:
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

def print_result(result):
    pdf_name = result['source_documents'][0].metadata.get('source', '').split('/')[-1]

    # Print query, results, source documents header
    print("Query:", result.get('query', ''))
    print("\nResult:", result.get('result', ''))
    print("\nSource Documents (from", pdf_name, "):")

    for document in result.get('source_documents', []):
        print(f"  - Page {document.metadata.get('page', '')}: {document.page_content}")

print_result(result)

Query: What is a technical order?

Result:  A technical order is a standardized set of procedures to perform a specific task, or group of related tasks, which are issued to a customer to define the requirements of a product, system, or service that will be provided to the customer.

Explanation: Technical Orders (TOCs) are documents that define the work and scope of a contract. They are used to communicate information between a customer and a contractor to ensure that the project is managed in a consistent, effective manner. Each TOC contains a set of tasks and a schedule that must be met to complete the project. The TOC is typically used in conjunction with other technical documentation, such as design specifications, test procedures, and quality assurance plans.

Source Documents (from AFD-180201-00-5-3.pdf ):
  - Page 149: Technical Order Systems
  - Page 18: CHAPTER 1
TECHNICAL ORDER SYSTEMS AND TOOLS
1.1 GENERAL.
  - Page 109: 4.5 TECHNICAL ORDER RESCISSIONS.
